In [202]:
import numpy as np
import math
from sklearn.model_selection import KFold
import random

In [203]:
#Read the data
file=open('car.data','r')
rows=file.read()
rows=rows.split('\n')
#deleting unwanted dat strings
rows.pop()
print(rows[0:10])
print('data has been loaded successfully')

['vhigh,vhigh,2,2,small,low,unacc', 'vhigh,vhigh,2,2,small,med,unacc', 'vhigh,vhigh,2,2,small,high,unacc', 'vhigh,vhigh,2,2,med,low,unacc', 'vhigh,vhigh,2,2,med,med,unacc', 'vhigh,vhigh,2,2,med,high,unacc', 'vhigh,vhigh,2,2,big,low,unacc', 'vhigh,vhigh,2,2,big,med,unacc', 'vhigh,vhigh,2,2,big,high,unacc', 'vhigh,vhigh,2,4,small,low,unacc']
data has been loaded successfully


In [204]:
# initialising list for pre-defined values for indices
buying=['vhigh','high','med','low']
doors=['2','3','4','5more']
persons=['2','4','more']
lug_boot=['small','med','big']
safety=['low','med','high']
cls=['unacc','acc','good','vgood']
dataset=[]

In [205]:
def data_preprocess(row):
    row_data=list(row.split(','))
    row_data=list(row.split(','))
    row_data[0]=buying.index(row_data[0])+1
    row_data[1]=buying.index(row_data[1])+1
    row_data[2]=doors.index(row_data[2])+1
    row_data[3]=persons.index(row_data[3])+1
    row_data[4]=lug_boot.index(row_data[4])+1
    

    if row_data[5] !='?':
        
        row_data[5]=safety.index(row_data[5])+1
    else:
        row_data[5]=0
        
    row_data[6]=cls.index(row_data[6])+1
    
    return row_data

In [206]:
#preprocessing the data by row
dataset = []
for row in rows:
    dataset.append(data_preprocess(row))
dataset[0:10]

[[1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 2, 1],
 [1, 1, 1, 1, 1, 3, 1],
 [1, 1, 1, 1, 2, 1, 1],
 [1, 1, 1, 1, 2, 2, 1],
 [1, 1, 1, 1, 2, 3, 1],
 [1, 1, 1, 1, 3, 1, 1],
 [1, 1, 1, 1, 3, 2, 1],
 [1, 1, 1, 1, 3, 3, 1],
 [1, 1, 1, 2, 1, 1, 1]]

In [207]:
#separate data by class
def separate_data_by_class(dataset):
    data_seg = {}
    for dt in dataset: 
        cls = dt[-1]
        if cls not in data_seg:
            data_seg[cls] = []
        data_seg[cls].append(dt)
    return data_seg

In [208]:
#data summarization
def summarize_data(dataset):
    summary =[(np.mean(col),np.std(col),len(col)) for col in zip(*dataset)]
    del summary[-1]
    return summary

In [209]:
#data summarization by class
def summarizing_data_by_cls(dataset):
    data_seg = separate_data_by_class(dataset)
    data_summaries = {}
    for cls ,row in data_seg.items():
        data_summaries[cls] = summarize_data(row)
    return data_summaries

In [210]:
#gaussian implementation
def gaussian(x,mean,std):
    try:
        gaussian=(1/(std*math.sqrt(2*math.pi)))*(math.exp(-(x-mean)**2/(2*std**2)))
        return gaussian
    except:
        return 0.01

In [211]:
#gaussian by class
def gaussian_by_cls(data_summaries,row):
    probs={}
    no_of_rows= sum([data_summaries[x][0][2] for x in data_summaries])
    for cls,cls_summaries in data_summaries.items():
        probs[cls]= 1
        for i in range(len(cls_summaries)):
            probs[cls]+=math.log(gaussian(row[i],cls_summaries[i][0],cls_summaries[i][1]))
    return probs


In [212]:
# predicting cls row wise
def predict(cls_summaries, row):
    probs=gaussian_by_cls(cls_summaries,row)
    best_cls, best_prob = None ,-1
    for cls, prob in probs.items():
        if best_cls==None or prob>best_prob:
            best_cls,best_prob=cls,prob 
    return best_cls

In [213]:

#acuuracy of the value
def accuracy(actual,predict):
    crct_count= len([i for i,j in zip(actual,predict) if i==j])
    acc=crct_count/float(len(actual))*100
    return acc

In [214]:
#navie_bayes
def navie_bayes(train, test):
    cls_summaries = summarizing_data_by_cls(train)
    predictions=[]
    for r in test:
        predictions.append(predict(cls_summaries,row))
    return predictions

In [215]:
#K_folds
def K_fold_splits(dataset, no_of_splits):
    size_of_fold=int(len(dataset)/no_of_splits)
    copy_dataset= list(dataset)
    folds=[]
    for _ in range(no_of_splits):
        fold=[]
        while len(fold)<size_of_fold:
            r_idx= random.randrange(len(copy_dataset))
            fold.append(copy_dataset.pop(r_idx))
        folds.append(fold)
    return folds

In [201]:
print('dataset length:',len(dataset))
no_of_folds =10
print('In',no_of_folds,'folds test')
folds =K_fold_splits(dataset, no_of_folds)
accuracies =[]
for fold in folds:
    train = list(folds)
    train.remove (fold)
    train_set = []
    for idx in range(len(train)):
        train_set= train_set + train[idx]
    actual_set= []
    for row in fold:
        actual_set.append(row[-1])
    test_set=[]
    for row in fold:
        test_set.append(row)
    print('length of train data:',len(train_set))
    print('length of test data:',len(test_set))
    predict_set =  navie_bayes(train_set, test_set)
    accuracies.append(accuracy(actual_set,predict_set))
print('model_accuracy:', np.mean(accuracies))

dataset length: 1728
In 10 folds test
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
length of train data: 1548
length of test data: 172
model_accuracy: 35.75581395348837


C:\Users\vamse\AppData\Local\Temp/ipykernel_10892/3554333303.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  gaussian=(1/(std*math.sqrt(2*math.pi)))*(math.exp(-(x-mean)**2/(2*std**2)))
C:\Users\vamse\AppData\Local\Temp/ipykernel_10892/3554333303.py:4: RuntimeWarning: invalid value encountered in double_scalars
  gaussian=(1/(std*math.sqrt(2*math.pi)))*(math.exp(-(x-mean)**2/(2*std**2)))
